# Setting up env

## Importing Libs

In [3]:
import polars as pl
import pandas as pd
import yfinance as yf

## setting up paths

In [1]:
from paths import DATASET_OUTPUT_FILE, PRE_DATASET_FILE

## parameters

In [4]:
offset_days = 63

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
# Charger le dataset
df = pl.read_parquet(DATASET_OUTPUT_FILE)

print(df.shape)
print(df.head(5))

(9686, 8)
shape: (5, 8)
┌────────────┬─────────┬───────────┬─────────────────┬────────────────┬─────────┬───────────┬──────┐
│ date       ┆ cik     ┆ file_type ┆ rf              ┆ mgmt           ┆ gvkey   ┆ cusip     ┆ year │
│ ---        ┆ ---     ┆ ---       ┆ ---             ┆ ---            ┆ ---     ┆ ---       ┆ ---  │
│ date       ┆ i64     ┆ str       ┆ str             ┆ str            ┆ f64     ┆ str       ┆ i64  │
╞════════════╪═════════╪═══════════╪═════════════════╪════════════════╪═════════╪═══════════╪══════╡
│ 2006-02-09 ┆ 827054  ┆ 10Q       ┆ Item            ┆ Item 2.        ┆ 27965.0 ┆ 595017104 ┆ 2006 │
│            ┆         ┆           ┆ 1A. Risk        ┆ Management s   ┆         ┆           ┆      │
│            ┆         ┆           ┆ Factors         ┆ Discussio…     ┆         ┆           ┆      │
│            ┆         ┆           ┆                 ┆                ┆         ┆           ┆      │
│            ┆         ┆           ┆ 28 It…          ┆             

In [6]:
# Convertir ton df polars en pandas
pdf = df.to_pandas()
pdf["date"] = pd.to_datetime(pdf["date"])

# Télécharger l'historique du S&P500 symbol (ou individuellement pour chaque stock)
tickers = pdf["Symbol"].unique().tolist()

In [7]:
min_date = pdf["date"].min()
max_date = pdf["date"].max() + pd.DateOffset(days=offset_days)

print(f"min date :", min_date)
print(f"max date :", max_date)


min date : 2006-02-09 00:00:00
max date : 2025-08-01 00:00:00


In [8]:
# télécharger toutes les séries d’un coup
data = yf.download(
    tickers,
    start=min_date,
    end=max_date
)['Close']

print(data.shape)
print(data.head())

[*********************100%***********************]  168 of 168 completed

1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


(4899, 168)
Ticker            AEE        AIZ       AKAM        ALB  ALGN        AMD  \
Date                                                                      
2006-02-09  22.618095  30.220770  26.010000  16.064569  8.41  40.060001   
2006-02-10  22.748232  30.728865  25.540001  16.248652  8.25  40.099998   
2006-02-13  22.707853  30.088528  24.930000  16.060806  8.12  38.959999   
2006-02-14  22.434097  30.784550  25.100000  16.203571  8.17  40.220001   
2006-02-15  22.461029  30.659269  25.770000  16.143459  8.33  40.240002   

Ticker           AMGN        AMP    AMZN       APH  ...        WAB        WEC  \
Date                                                ...                         
2006-02-09  48.708122  29.673944  1.8995  2.681131  ...  14.343351  10.898447   
2006-02-10  49.009937  29.437487  1.9260  2.675675  ...  14.208611  10.825067   
2006-02-13  48.797287  28.971292  1.8930  2.644029  ...  14.064572  10.871270   
2006-02-14  49.339188  29.687469  1.9125  2.613475  ...  

In [9]:
pdf.reset_index(inplace = True)
pdf["date"] = pd.to_datetime(pdf["date"])   # conversion en datetime
pdf = pdf.set_index(["Symbol", "date"])                 # nouvel index

In [10]:
print(pdf)
print(pdf.index)

                   index      cik file_type  \
Symbol date                                   
MCHP   2006-02-09      0   827054       10Q   
RL     2006-02-09      1  1037038       10Q   
MKC    2006-04-06      2    63754       10Q   
QCOM   2006-04-19      3   804328       10Q   
DHR    2006-04-20      4   313616       10Q   
...                  ...      ...       ...   
WSM    2025-05-28   9739   719955       10Q   
SNPS   2025-05-28   9740   883241       10Q   
LOW    2025-05-29   9741    60667       10Q   
CRM    2025-05-29   9742  1108524       10Q   
TGT    2025-05-30   9743    27419       10Q   

                                                                  rf  \
Symbol date                                                            
MCHP   2006-02-09  Item\n 1A. Risk Factors \n\n28 Item 1A. Risk\n...   
RL     2006-02-09  Item 1A. \n\nRisk Factors \n\n37 Item 1A. Risk...   
MKC    2006-04-06  ITEM 1A \n\nRISK FACTORS ITEM 1A. RISK FACTORS...   
QCOM   2006-04-19  Item 1A. 

In [11]:
print(data)
print(data.index)

Ticker             AEE         AIZ       AKAM        ALB        ALGN  \
Date                                                                   
2006-02-09   22.618095   30.220770  26.010000  16.064569    8.410000   
2006-02-10   22.748232   30.728865  25.540001  16.248652    8.250000   
2006-02-13   22.707853   30.088528  24.930000  16.060806    8.120000   
2006-02-14   22.434097   30.784550  25.100000  16.203571    8.170000   
2006-02-15   22.461029   30.659269  25.770000  16.143459    8.330000   
...                ...         ...        ...        ...         ...   
2025-07-25   99.389221  185.588821  80.750000  83.288101  205.559998   
2025-07-28   97.621857  184.164139  79.709999  74.366508  207.190002   
2025-07-29   99.111206  184.492905  79.489998  71.213615  205.809998   
2025-07-30   99.131065  183.875214  78.059998  68.518242  203.570007   
2025-07-31  100.411903  186.605042  76.309998  67.483849  129.009995   

Ticker             AMD        AMGN         AMP        AMZN     

In [12]:
print(data.index)     # DatetimeIndex
print(data.columns)   # Index des tickers

DatetimeIndex(['2006-02-09', '2006-02-10', '2006-02-13', '2006-02-14',
               '2006-02-15', '2006-02-16', '2006-02-17', '2006-02-21',
               '2006-02-22', '2006-02-23',
               ...
               '2025-07-18', '2025-07-21', '2025-07-22', '2025-07-23',
               '2025-07-24', '2025-07-25', '2025-07-28', '2025-07-29',
               '2025-07-30', '2025-07-31'],
              dtype='datetime64[ns]', name='Date', length=4899, freq=None)
Index(['AEE', 'AIZ', 'AKAM', 'ALB', 'ALGN', 'AMD', 'AMGN', 'AMP', 'AMZN',
       'APH',
       ...
       'WAB', 'WEC', 'WELL', 'WFC', 'WM', 'WRB', 'WSM', 'YUM', 'ZBH', 'ZBRA'],
      dtype='object', name='Ticker', length=168)


In [13]:
forward_returns = data.shift(-63) / data - 1

In [14]:
print(forward_returns.head())
print(forward_returns.tail())
print(forward_returns.shape)

Ticker           AEE       AIZ      AKAM       ALB      ALGN       AMD  \
Date                                                                     
2006-02-09 -0.009798  0.153639  0.339485  0.142229 -0.051130 -0.213929   
2006-02-10 -0.028649  0.135244  0.346515  0.113969 -0.060606 -0.210474   
2006-02-13 -0.016514  0.175858  0.302046  0.117134 -0.043103 -0.201232   
2006-02-14 -0.012819  0.151311  0.296813  0.100541 -0.044064 -0.233963   
2006-02-15 -0.024929  0.129179  0.242918  0.074500 -0.068427 -0.235338   

Ticker          AMGN       AMP      AMZN       APH  ...       WAB       WEC  \
Date                                                ...                       
2006-02-09 -0.052528  0.103365 -0.117399  0.182432  ...  0.236787  0.007238   
2006-02-10 -0.046746  0.092675 -0.150312  0.144239  ...  0.202395  0.006238   
2006-02-13 -0.018414  0.119608 -0.134179  0.156902  ...  0.212066  0.018556   
2006-02-14 -0.035312  0.080052 -0.143529  0.167294  ...  0.192217  0.001468   
2006-02

In [15]:
forward_returns = forward_returns.iloc[:-63]

In [16]:
print(forward_returns.head())
print(forward_returns.tail())
print(forward_returns.shape)

Ticker           AEE       AIZ      AKAM       ALB      ALGN       AMD  \
Date                                                                     
2006-02-09 -0.009798  0.153639  0.339485  0.142229 -0.051130 -0.213929   
2006-02-10 -0.028649  0.135244  0.346515  0.113969 -0.060606 -0.210474   
2006-02-13 -0.016514  0.175858  0.302046  0.117134 -0.043103 -0.201232   
2006-02-14 -0.012819  0.151311  0.296813  0.100541 -0.044064 -0.233963   
2006-02-15 -0.024929  0.129179  0.242918  0.074500 -0.068427 -0.235338   

Ticker          AMGN       AMP      AMZN       APH  ...       WAB       WEC  \
Date                                                ...                       
2006-02-09 -0.052528  0.103365 -0.117399  0.182432  ...  0.236787  0.007238   
2006-02-10 -0.046746  0.092675 -0.150312  0.144239  ...  0.202395  0.006238   
2006-02-13 -0.018414  0.119608 -0.134179  0.156902  ...  0.212066  0.018556   
2006-02-14 -0.035312  0.080052 -0.143529  0.167294  ...  0.192217  0.001468   
2006-02

In [17]:
print(forward_returns.index.name)     # 'Date'
print(forward_returns.columns.name)   # 'Ticker'

Date
Ticker


In [18]:
dates_pdf = pdf.index.get_level_values("date").unique()

In [19]:
dates_pdf

DatetimeIndex(['2006-02-09', '2006-04-06', '2006-04-19', '2006-04-20',
               '2006-04-21', '2006-04-24', '2006-04-25', '2006-04-26',
               '2006-04-27', '2006-04-28',
               ...
               '2025-05-05', '2025-05-06', '2025-05-07', '2025-05-08',
               '2025-05-09', '2025-05-12', '2025-05-14', '2025-05-28',
               '2025-05-29', '2025-05-30'],
              dtype='datetime64[ms]', name='date', length=1546, freq=None)

In [20]:
# méthode filtre direct
data_filtered = forward_returns.loc[forward_returns.index.intersection(dates_pdf)]

In [21]:
data_filtered = forward_returns.reindex(dates_pdf)

In [22]:
print(data_filtered)

Ticker           AEE       AIZ      AKAM       ALB      ALGN       AMD  \
date                                                                     
2006-02-09 -0.009798  0.153639  0.339485  0.142229 -0.051130 -0.213929   
2006-04-06  0.025037 -0.011264  0.108839  0.038767 -0.191919 -0.321037   
2006-04-19  0.032883 -0.016806 -0.014063  0.020769 -0.281513 -0.319594   
2006-04-20  0.039663 -0.015417 -0.051802 -0.021595 -0.300211 -0.310070   
2006-04-21  0.040920 -0.023466 -0.128251 -0.025878 -0.288525 -0.424519   
...              ...       ...       ...       ...       ...       ...   
2025-05-12       NaN       NaN       NaN       NaN       NaN       NaN   
2025-05-14       NaN       NaN       NaN       NaN       NaN       NaN   
2025-05-28       NaN       NaN       NaN       NaN       NaN       NaN   
2025-05-29       NaN       NaN       NaN       NaN       NaN       NaN   
2025-05-30       NaN       NaN       NaN       NaN       NaN       NaN   

Ticker          AMGN       AMP      A

In [23]:
nan_dates = data_filtered.index[data_filtered.isna().all(axis=1)]
print(nan_dates)

DatetimeIndex(['2007-04-06', '2011-04-22', '2012-10-29', '2012-10-30',
               '2019-04-19', '2025-04-18', '2025-05-01', '2025-05-02',
               '2025-05-05', '2025-05-06', '2025-05-07', '2025-05-08',
               '2025-05-09', '2025-05-12', '2025-05-14', '2025-05-28',
               '2025-05-29', '2025-05-30'],
              dtype='datetime64[ms]', name='date', freq=None)


In [24]:
# Ton premier dataset de returns : index = date, colonnes = tickers
returns = data_filtered.copy()  

# Transforme en long format pour avoir (Symbol, date, return)
returns_long = returns.reset_index().melt(
    id_vars="date", var_name="Symbol", value_name="return"
)

# Mets le même index que le deuxième dataset
returns_long = returns_long.set_index(["Symbol", "date"])

# Merge avec le deuxième dataset
merged = pdf.join(returns_long, how="left")

print(merged.head())

                   index      cik file_type  \
Symbol date                                   
MCHP   2006-02-09      0   827054       10Q   
RL     2006-02-09      1  1037038       10Q   
MKC    2006-04-06      2    63754       10Q   
QCOM   2006-04-19      3   804328       10Q   
DHR    2006-04-20      4   313616       10Q   

                                                                  rf  \
Symbol date                                                            
MCHP   2006-02-09  Item\n 1A. Risk Factors \n\n28 Item 1A. Risk\n...   
RL     2006-02-09  Item 1A. \n\nRisk Factors \n\n37 Item 1A. Risk...   
MKC    2006-04-06  ITEM 1A \n\nRISK FACTORS ITEM 1A. RISK FACTORS...   
QCOM   2006-04-19  Item 1A. Risk Factors \n\n52 ITEM 1A. RISK FAC...   
DHR    2006-04-20  Item 1A. \n\nRisk Factors \n \n 30 ITEM 1A. RI...   

                                                                mgmt    gvkey  \
Symbol date                                                                     
MCH

In [32]:
print(merged.columns)
merged = merged[["mgmt", "rf", "return"]]
print(merged.columns)

Index(['mgmt', 'rf', 'return'], dtype='object')
Index(['mgmt', 'rf', 'return'], dtype='object')


In [33]:
cleaned = merged.dropna(subset=["return", "mgmt", "rf"])

In [34]:
print("Avant :", len(merged))
print("Après :", len(cleaned))

Avant : 9744
Après : 9516


In [35]:
print(cleaned.shape)

(9516, 3)


In [36]:
cleaned.to_parquet(PRE_DATASET_FILE)